In [2]:
library("plotgardener")
library("data.table")
library("org.Hs.eg.db")
library("TxDb.Hsapiens.UCSC.hg38.knownGene")
library(png)
options(bedtools.path = "/rds/general/user/hrayjone/home/anaconda3/bin/")
library(bedtoolsr)

In [3]:
loops <- fread("~/HRJ_monocytes/CHiC/chicago/combinations_output/DpnII/combined_34reps_DpnII_binsize1500_maxL150K_mergeWeights/data/combined_34reps_DpnII_binsize1500_maxL150K_mergeWeights.bedpe")
# intersect loops with proxies so we can restrict loops to just incldue them

proxies1 <- fread("~/HRJ_monocytes/findmotifs/find_new_dpnII/snps_went_into_loopingQTL_analysis.DpnII.marked.REFALTseqs.PURGED.DISTAL.txt")
proxies1[, chrom := paste0("chr", Chr)]
proxies1[, hg38Proxy_pos0 := hg38Proxy_pos-1]
proxies_small <- proxies1[, .(chrom, hg38Proxy_pos0, hg38Proxy_pos, hg38Proxy_ID, hg38SNP_ID)]
setkey(loops, V1, V2, V3)
loops1 <- foverlaps(proxies_small, loops, by.x = c("chrom", "hg38Proxy_pos0", "hg38Proxy_pos"), nomatch = NULL)
setnames(loops1, c("chrom", "V2", "V3", "V4", "V5", "V6"), c("chrom1", "start1", "end1", "chrom2", "start2", "end2"))
setkey(loops, V4, V5, V6)
loops2 <- foverlaps(proxies_small, loops, by.x = c("chrom", "hg38Proxy_pos0", "hg38Proxy_pos"), nomatch = NULL)
setnames(loops2, c("chrom", "V1", "V2", "V3", "V5", "V6"), c("chrom2", "chrom1", "start1", "end1", "start2", "end2"))
loops2_edit <- loops2[, .(chrom1, start1, end1, chrom2, start2, end2, hg38Proxy_pos0, hg38Proxy_pos, hg38Proxy_ID, hg38SNP_ID)]
loops_proxies <- rbind(loops1, loops2_edit)

## OR: we can instead look at eQTLs and only their interactions with eGenes
eqtl_inters <- fread("~/HRJ_monocytes/CHiC/chicago/findings/consensus_DpnII_level_proxy_eGene_interactions_DpnIIFilt_withGeneTypes.txt")
sig_eqtl_inters <- eqtl_inters[score >= 5]
sig_eqtl_inters <- sig_eqtl_inters[, c("Chr", "Proxy_dpnStart", "Proxy_dpnEnd", "DpnID", "Gene", "hg19SNP_ID", "hg19Proxy_ID", "hg38SNP_ID")]
# have to get other end coordinates
dpnII <- fread("/rds/general/project/lms-spivakov-analysis/live/Design/Human_eQTL_CHiC_DpnII_hg38/hg38_dpnII.rmap")
setkey(dpnII, V4)
sig_eqtl_inters_dpn <- sig_eqtl_inters[dpnII, on = c(DpnID = "V4"), nomatch = NULL]
setnames(sig_eqtl_inters_dpn, c("V2", "V3"), c("TSS_dpnStart", "TSS_dpnEnd"))
sig_eqtl_inters_dpn[, chrom1 := paste0("chr", Chr)]
sig_eqtl_inters_dpn[, chrom2 := paste0("chr", V1)]
eQTL_eGene_score5 <- sig_eqtl_inters_dpn[, .(chrom1, Proxy_dpnStart, Proxy_dpnEnd, chrom2, TSS_dpnStart, TSS_dpnEnd, Gene, hg19SNP_ID, hg19Proxy_ID, hg38SNP_ID)]


# need to make proxy_sets only the ones that we keep!
proxy_sets <- fread("~/HRJ_monocytes/Browser_tracks/proxy_setshg38.bed")
eqtls <- fread("~/HRJ_monocytes/Browser_tracks/eqtlshg38.bed")
#proxies <- fread("~/HRJ_monocytes/Browser_tracks/proxieshg38.bed")
proxies <- fread("~/HRJ_monocytes/findmotifs/find_new_dpnII/snps_went_into_loopingQTL_analysis.DpnII.marked.REFALTseqs.PURGED.DISTAL.txt")
proxies[, chrom := paste0("chr", Chr)]
proxies_plot <- proxies[, .(chrom, hg38Proxy_pos, hg38Proxy_end, hg38SNP_ID)]

# Load in DpnII baits
baits <- fread("/rds/general/project/lms-spivakov-analysis/live/Design/Human_eQTL_CHiC_DpnII_hg38/eCHiC_grch38.baitmap")
baits[, chrom := paste0("chr", V1)]
baits <- baits[, .(chrom, V2, V3, V4, V5)]
names(baits) = c("chrom", "DpnStart", "DpnEnd", "DpnID", "DpnName")

# Bring in data for plotBaits: bg file. NO, too big!
#int <- fread("~/HRJ_monocytes/CHiC/chicago/combinations_output/DpnII/combined_34reps_DpnII_binsize1500_maxL150K_mergeWeights/data/combined_34reps_DpnII_binsize1500_maxL150K_mergeWeights_interactions.txt")

# also make an overlap with genes
proxies1[, TSS1 := TSS - 500]
proxies1[, TSS2 := TSS + 500]
genes_small <- proxies1[, .(chrom, TSS1, TSS2, rep_Gene, Gene, ENSG_ID)]
setkey(loops, V1, V2, V3)
loopsG1 <- foverlaps(genes_small, loops, by.x = c("chrom", "TSS1", "TSS2"), nomatch = NULL)
setnames(loopsG1, c("chrom", "V2", "V3", "V4", "V5", "V6"), c("chrom1", "start1", "end1", "chrom2", "start2", "end2"))
setkey(loops, V4, V5, V6)
loopsG2 <- foverlaps(genes_small, loops, by.x = c("chrom", "TSS1", "TSS2"), nomatch = NULL)
setnames(loopsG2, c("chrom", "V1", "V2", "V3", "V5", "V6"), c("chrom2", "chrom1", "start1", "end1", "start2", "end2"))
loopsG2_edit <- loopsG2[, .(chrom1, start1, end1, chrom2, start2, end2, TSS1, TSS2, rep_Gene, Gene, ENSG_ID)]
loops_genes <- rbind(loopsG1, loopsG2_edit)

## bring in the regions that were tested in BaseQTL. for each proxy SNP, we can filter this file and merge the gene end to make one PE contact.
regions <- fread("/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/AS_CHiC/BaseQTL/scripts/bedpe_for_chic_egenes_with_Gene_number.txt")
regions[, chrom1 := paste0("chr", V1)]
regions[, chrom2 := paste0("chr", V4)]

In [4]:
### Read in the BaseQTL results, in order to plot the contact QTLs

BaseRes <- fread("~/HRJ_monocytes/BaseQTL/findings_round2_dpnIICorrection/BaseQTL_CHiC_Filt_on_Dpn_Distal_Rhat_AI_rsids.txt")
BaseBed <- BaseRes[Signif.99 == "yes", .(chrom, hg38Proxy_pos, hg38Proxy_end, Gene, hg19SNP_ID, BaseQTL_SNP_hg19, rsid, log_mean_aFC)]


In [13]:
### Bring in ENCODE ccREs
encode <- fread("/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/external_data/ENCODE/GRCh38-cCREs.bed")
#unique(encode$V6)
ELS <- encode[V6 %like% "ELS"]
mono_Enh <- fread("~/ext_datasets/blueprint_projected_segmentations/hg38/active_enhancers_mon.hg38.bed")
mono_Enh


V1,V2,V3
<chr>,<int>,<int>
chr1,270600,271490
chr1,274244,275580
chr1,844000,844800
chr1,898600,898800
chr1,1103800,1105200
chr1,1188800,1189800
chr1,1405525,1405725
chr1,1748598,1749567
chr1,1806916,1807525


In [6]:
#### Prepare genotype-specific PCK2 data - NOT USING NOW, BUT DO GET LOOPS
mydir <- "~/HRJ_monocytes/CHiC/merge_genos/chicago/output/DpnII/data"
source("~/Rfunctions/helen_functions.R")


#ref_eQTL <- fread(paste(mydir, "14:24527672:G:A_homozygous_REF_merged_DpnII_washU_NEW_eQTLinteractions.txt", sep = "/"))
#ref_eQTL_bpe <- washuNEW2bedpe(ref_eQTL)
#ref_eQTL_bpe[, genotype_REF := "TRUE"]

#alt_eQTL <- fread(paste(mydir, "14:24527672:G:A_homozygous_ALT_merged_DpnII_washU_NEW_eQTLinteractions.txt", sep = "/"))
#alt_eQTL_bpe <- washuNEW2bedpe(alt_eQTL)
#alt_eQTL_bpe[, genotype_ALT := "TRUE"]

#both_eQTL_bpe <- merge.data.table(ref_eQTL_bpe, alt_eQTL_bpe, on = c("V1", "V2", "V3", "chr2", "start2", "end2"), all = T)
#both_eQTL_bpe[genotype_REF == T & genotype_ALT == T, allele := "Either"]
#both_eQTL_bpe[genotype_REF == T & is.na(genotype_ALT), allele := "REF"]
#both_eQTL_bpe[is.na(genotype_REF) & genotype_ALT == T, allele := "ALT"]
#both_eQTL_bpe

#ref_PCK2 <- fread(paste(mydir, "14:24527672:G:A_homozygous_REF_merged_DpnII_washU_NEW_PCK2_allPromInteractions.txt", sep = "/"))
#ref_PCK2_bpe <- washuNEW2bedpe(ref_PCK2)
#ref_PCK2_bpe[, genotype_REF := "TRUE"]

#alt_PCK2 <- fread(paste(mydir, "14:24527672:G:A_homozygous_ALT_merged_DpnII_washU_NEW_PCK2_allPromInteractions.txt", sep = "/"))
#alt_PCK2_bpe <- washuNEW2bedpe(alt_PCK2)
#alt_PCK2_bpe[, genotype_ALT := "TRUE"]

#both_PCK2_bpe <- merge.data.table(ref_PCK2_bpe, alt_PCK2_bpe, on = c("V1", "V2", "V3", "chr2", "start2", "end2"), all = T)
#both_PCK2_bpe[genotype_REF == T & genotype_ALT == T, allele := "Either"]
#both_PCK2_bpe[genotype_REF == T & is.na(genotype_ALT), allele := "REF"]
#both_PCK2_bpe[is.na(genotype_REF) & genotype_ALT == T, allele := "ALT"]
#both_PCK2_bpe

# Reading interactions from the chicdiff analysis
mydir <- "~/HRJ_monocytes/CHiC/merge_genos/chicdiff"
#list.files(mydir)
PCK2counts <- readRDS(paste0(mydir, "/PCK2_14:24527672:G:A_5K_expand3_countput.Rds"))
rmap5K <- fread("/rds/general/project/lms-spivakov-analysis/live/Design/Human_eQTL_CHiC_DpnII_hg38_5K_bins/human_DpnII_5K.rmap")
names(rmap5K) = c("chr", "start", "end", "ID")
rmap5K[, chrom := paste0("chr", chr)]
rmap5K[, chr := NULL]
setkey(rmap5K, ID)

sigLoops <- PCK2counts[score > 5]
loops1 <- rmap5K[sigLoops, on = c(ID = "baitID")]
setnames(loops1, c("chrom", "start", "end", "ID"), c("baitChrom", "baitStart", "baitEnd", "baitID"))
loops2 <- rmap5K[loops1, on = c(ID = "otherEndID")]
setnames(loops2, c("chrom", "start", "end", "ID"), c("oeChrom", "oeStart", "oeEnd", "oeID"))
loops2[baitStart > oeStart, `:=` (leftChr = oeChrom, 
                                  leftStart = oeStart, 
                                  leftEnd = oeEnd, 
                                 rightChr = baitChrom, 
                                 rightStart = baitStart, 
                                 rightEnd = baitEnd)]
loops2[baitStart < oeStart, `:=` (leftChr = baitChrom, 
                                  leftStart = baitStart, 
                                  leftEnd = baitEnd, 
                                 rightChr = oeChrom, 
                                 rightStart = oeStart, 
                                 rightEnd = oeEnd)]
loops_plot <- loops2[, .(leftChr, leftStart, leftEnd, rightChr, rightStart, rightEnd, condition, baitID)]


Plot the PCK2 locus including loops for different genotypes. Will insert the plotbaits from chicdiff. Make sure to use 200kb to left and right of pck2.

mid of the bait 135210 at PCK2 is 24095221, so start = 23895221 and end = 24295221

In [7]:
myBait <- 135210
rmap5K[ID == myBait]
rmap5K[, mid := (start+end)/2]
rmap5K[ID == myBait]
24095221 - 200000
24095221 + 200000

start,end,ID,chrom
<int>,<int>,<int>,<chr>
24092611,24097831,135210,chr14


start,end,ID,chrom,mid
<int>,<int>,<int>,<chr>,<dbl>
24092611,24097831,135210,chr14,24095221


[1] 23895221

[1] 24295221

In [24]:
#### PCK2 locus: different genotypes
#### check colorbind colors here check colours here https://colorbrewer2.org/#type=diverging&scheme=RdYlBu&n=7


### set parameters
myeQTL <- "14:24058463:G:A"
myGene = "PCK2"
mychrom = "chr14"
mystart = 23895221
myend = 24295221
###
### restrict loops to PCK2
REF_loops <- unique(loops_plot[condition == "REF_5K" & baitID == myBait])
ALT_loops <- unique(loops_plot[condition == "ALT_5K" & baitID == myBait])

###
### restrict eQTLs to the ones we are intersted in?
myeqtls <- eqtls[V4 %like% myeQTL]
### restrict proxies to the ones we are interested in
myproxies <- proxies_plot[hg38SNP_ID == myeQTL]

# get CTCF predicted mono
mybeds <- "/rds/general/user/hrayjone/home/ext_datasets/blueprint_projected_segmentations/hg38"
CTCF <- fread(paste(mybeds, "CTCF_monocyte.bed", sep = "/"))

pdf(file = "~/HRJ_monocytes/paper/PCK2_refalt_plotgardener_genoSpecific_V02.pdf")
pageCreate(width = 15, height = 12, default.units = "cm", showGuides = F)
########## Set parameters
params_main <- pgParams(chrom = mychrom, chromstart = mystart, chromend = myend, 
                      assembly = "hg38", , default.units = "cm",
                      x = 2, width = 10, just = c("left", "top"), default.units = "cm")

params_text <- pgParams(chrom = mychrom, chromstart = mystart, chromend = myend, 
                      assembly = "hg38", , default.units = "cm",
                      x = 1.3, width = 10, just = c("right", "top"), default.units = "cm")
##########
## specify the gene target
geneHighlights <- data.frame("geneName" = myGene, "color" = "chartreuse4")

## Plot gene track
genes_a <- plotGenes(params = params_main, stroke = 1, fontsize = 8,
                     geneHighlights = geneHighlights, geneBackground = "grey48",
                        y = 3.8, height = 1)

# text for genes
plotText(label = "Genes", fontsize = 7, fontcolor = "grey48", rot = 0, params = params_main,
    y = 4.3, x = 0.4, height = 1,
) 

## Annotate genome label
annoGenomeLabel(plot = genes_a, params = params_main, 
                   scale = "Mb", fontsize = 7,
                   y = 10.3)

#### Add genotype specific 5Kb plotBaits NOT DOING NOW. PASTED BELOW.

# Plot eQTLs
#plotRanges( 
#    data = myeqtls, params = params_main, linecolor="fill", fill = "gray40", collapse = T,
#    y = 1.2, height = 0.3
#)
# text for eQTLs
#plotText(
#    label = "Lead eQTL", 
#    fontsize = 7, fontcolor = "gray40", rot = 0, params = params_text, y = 1.2, height = 0.5,
#)

### Plot the contact QTLs
contact <- BaseBed[Gene == myGene]
plotRanges( 
    data = contact, params = params_main, linecolor="tomato3", fill = "tomato3", collapse = T,
    y = 2.8, height = 0.3
)
# text for contact QTLs
plotText(
    label = "Contact eQTL/ \nlead eQTL", 
    fontsize = 7, fontcolor = "tomato3", rot = 0, params = params_text, y = 2.8, height = 0.5,
)

# Plot proxies
plotRanges( 
    data = myproxies, params = params_main, linecolor="fill", fill = "gray40", collapse = T,
    y = 3.3, height = 0.3
)
# text for proxies
plotText(
    label = "LD SNPs", 
    fontsize = 7, fontcolor = "gray40", rot = 0, params = params_text, y = 3.5, height = 0.5
)
## plot ATAC
atac_myregion <- readBigwig(file = "/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/ATAC-seq/analyses/Genrich/consensus/consensus_Genrich.bw",
                        chrom = mychrom,
                        chromstart = mystart,
                        chromend = myend)
ATAC_signal <- plotSignal(
    data = atac_myregion,
    params = params_main, height = 1, y = 0,
    linecolor = "seagreen"
)
# text for ATAC
plotText(
    label = "ATAC \n(-log10P)", fontsize = 7, fontcolor = "seagreen",
    rot = 0,
    params = params_text,
    y = 0.2, height = 0.5,
) 

# range for ATAC signal
annoYaxis(
    ATAC_signal,
    at = c(0,13), label = T, # top ATAC -log10P
    main = TRUE, axisLine = T, params = params_main, 
    y = 0, height = 1, fontsize = 7, fontcolor = "seagreen"
)
##### plot loops.
plotPairsArches(
    data = REF_loops, params = params_main, y = 6.5, height = 1,
    fill = "#0072B2", linecolor = "fill", flip = TRUE, clip = T
)
# text for loops: REF
plotText(
    label = "PCK2 significant\ncontacts, REF", fontsize = 7,
    params = params_text,
    y = 6.5, height = 0.5, fontcolor = "#0072B2"
)
plotPairsArches(
    data = ALT_loops, params = params_main, y = 9, height = 1,
    fill = "#b2182b", linecolor = "fill", flip = TRUE, clip = T
)
# text for loops: ALT
plotText(
    label = "PCK2 significant\ncontacts, ALT", fontsize = 7,
    params = params_text,
    y = 9.1, height = 0.5, fontcolor = "#b2182b"
)

### put in text for plotbaits
plotText(
    label = "PCK2 bait plot at\n 5kb resolution, REF", fontsize = 7,
    params = params_text, x = 1, 
    y = 5.2, height = 0.5, fontcolor = "gray40"
)
plotText(
    label = "PCK2 bait plot at\n 5kb resolution, ALT", fontsize = 7,
    params = params_text, x = 1,
    y = 7.9, height = 0.5, fontcolor = "gray40"
)

### plot CTCF
## Load ENCODE ChIP-seq: reps 1 and 2 from accession ENCFF167VRL http://dcc.blueprint-epigenome.eu/#/files
CTCF_encode <- readBigwig(file = "/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/external_data/ENCODE/ENCFF167VRL_CTCF.bigWig", 
                          chrom = mychrom,
                          chromstart = mystart,
                          chromend = myend)
CTCFsignal <- plotSignal(
    data = CTCF_encode,
    params = params_main, height = 0.8, y = 1.2,
    linecolor = "maroon3"
)
# text for ENCODE CTCF ChIP seq
plotText(
    label = "ENCODE CTCF, \nmono (-log10P)", fontsize = 7, fontcolor = "maroon3",
    rot = 0,
    params = params_text,
    y = 1.2, height = 0.5,
) 

# range for CTCF
annoYaxis(
    CTCFsignal, at = c(0,478),
    label = T, # top -log10P val
    main = TRUE, axisLine = T, params = params_main, 
    y = 1.2, height = 0.8, fontsize = 7, fontcolor = "gray40"
)

## plot enhancers
encode <- plotRanges(
    data= mono_Enh, 
    params = params_main, fill = "burlywood", # only one type of cCRE in this region - distal enhancer like signature
    linecolor = "fill",
    collapse = T, y = 2.2, height = 0.3
)
# text for enhancers
plotText(
    label = "Blueprint projected\nmonocyte enhancers", fontsize = 7, fontcolor = "burlywood",
    rot = 0,
    params = params_text,
    y = 2, height = 0.5,
) 

## Going with ENCODE for CTCF, instead(above)
## Add CTCF - (no other enhaner regions found in mono)
#plotRanges( 
#    data = CTCF,
#    params = params_main, linecolor="blue", fill = "blue", 
#    collapse = T,
#    y = 6, height = 0.5
#)
# text for CTCF
#plotText(
#    label = "Blueprint \nCTCF, mono", fontsize = 7, fontcolor = "blue",
#    rot = 0,
#    params = params_text,
#    y = 6.2, height = 0.5,
#) 

# HIGHLIGHT OUR REGION: eQTL
annoHighlight(
    plot = CTCFsignal, chrom = mychrom, chromstart = 24054268, chromend = 24059365, # this is the 5Kb bait
    y = 0, height = 10.3, just = c("left", "top"), default.units = "cm"
)

# HIGHLIGHT OUR REGION: PCK2 gene
# chr14:24092611-24097831
annoHighlight(plot = CTCFsignal, chrom = "chr14", chromstart = 24092611, chromend = 24097831, # this is the 5kb bait
    y = 0, height = 10.3, just = c("left", "top"), default.units = "cm"
)

dev.off()

genes[genes1]

text[text1]

Warning message:
“Start label is rounded.”
Warning message:
“End label is rounded.”
genomeLabel[genomeLabel1]

ranges[ranges1]

text[text2]

ranges[ranges2]

text[text2]

signal[signal1]

text[text2]

yaxis[yaxis1]

arches[arches1]

text[text2]

arches[arches2]

text[text2]

text[text2]

text[text2]

signal[signal2]

text[text2]

yaxis[yaxis2]

ranges[ranges3]

text[text2]

highlight[highlight1]

highlight[highlight2]



png 
  2

In [41]:
REF_loops

leftChr,leftStart,leftEnd,rightChr,rightStart,rightEnd,condition
<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>
chr14,23971624,23976928,chr14,24092611,24097831,REF_5K
chr14,23976929,23982303,chr14,24092611,24097831,REF_5K
chr14,24046767,24054267,chr14,24092611,24097831,REF_5K
chr14,24054268,24059365,chr14,24092611,24097831,REF_5K
chr14,24059366,24066572,chr14,24092611,24097831,REF_5K
chr14,24035498,24041675,chr14,24092611,24097831,REF_5K
chr14,24092611,24097831,chr14,24133811,24139137,REF_5K
chr14,24092611,24097831,chr14,24178165,24183603,REF_5K
chr14,24092611,24097831,chr14,24193697,24198834,REF_5K


V1,V2,V3,V4
<chr>,<int>,<int>,<chr>
chr14,24058463,24058464,14:24058463:G:A


In [ ]:
### Make a plot of the region surrounding our SNP
### See if can add the motif as a zoom in to the SNP


mybeds <- "/rds/general/user/hrayjone/home/ext_datasets/blueprint_projected_segmentations/hg38"
#h3K27ac <- fread(paste(mybeds, "H3K27ac_monocyte.bed", sep = "/"))
#atacbed <- fread(paste(mybeds, "consensus_Genrich.bed", sep = "/"))
# no other segmentations here
CTCF <- fread(paste(mybeds, "CTCF_monocyte.bed", sep = "/"))
### set parameters
myeQTL <- "14:24058463:G:A"
myGene = "PCK2"
mychrom = "chr14"
mystart = 24058000
myend = 24059000
###
### the region is encode enhancer!! EH38E1703534 (enhancer-like signature, CTCF bound https://screen-v2.wenglab.org/search/?q=EH38E1703534&assembly=GRCh38)
encode <- fread("/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/external_data/ENCODE/GRCh38-cCREs.bed")

pdf(file = "~/HRJ_monocytes/paper/PCK2_SNP_zoomIn_plotgardener.pdf")
pageCreate(width = 13, height = 7, default.units = "cm", showGuides = F)
params_a <- pgParams(chrom = mychrom, chromstart = mystart, chromend = myend, 
                      assembly = "hg38", , default.units = "cm",
                      x = 2.5, width = 10, just = c("left", "top"), default.units = "cm")
## Plot gene track
genes_a <- plotGenes(params = params_a, stroke = 1, fontsize = 8, fontcolor = "grey48",
                     fill = "grey48",
                        y = 3.3, height = 1)
# text for genes
plotText(
    label = "Genes", fontsize = 7, fontcolor = "grey48",
    rot = 0,
    params = params_a,
    y = 3.8, x = 0.1, height = 0.5,
) 

# text for annotate genome
plotText(
    label = "chr14", fontsize = 7, fontcolor = "black",
    rot = 0,
    params = params_a,
    y = 4.2, x = 0.1, height = 0.5,
)

## Add eQTL
plotRanges( 
    data = proxy_sets_thisone,
    params = params_a, linecolor="tomato3", fill = "tomato3", 
    collapse = T,
    y = 0.5, height = 0.3
)
# text for eQTLs
plotText(
    label = "eQTLs", fontsize = 7, fontcolor = "tomato3",
    rot = 0,
    params = params_a,
    y = 0.6, x = 0.1, height = 0.5,
)

# text for rs7146599
plotText(
    label = "rs7146599", fontsize = 7, fontcolor = "tomato3",
    rot = 0,
    params = params_a,
    y = 0.5, x = 7.3, height = 0.5,
)



## Add CTCF - (no other enhaner regions found in mono)
plotRanges( 
    data = CTCF,
    params = params_a, linecolor="blue", fill = "blue", 
    collapse = T,
    y = 2.3, height = 0.2
)
# text for CTCF
plotText(
    label = "Blueprint \nCTCF, mono", fontsize = 7, fontcolor = "blue",
    rot = 0,
    params = params_a,
    y = 2, x = 0.1, height = 0.5,
) 


## Add ENCODE ccREs
plotRanges( 
    data = encode,
    params = params_a,
    fill = "burlywood", # only one type of cCRE in this region - distal enhancer like signature
    linecolor = "fill",
    collapse = T,
    y = 2.8, height = 0.2
)
# text for enhancer
plotText(
    label = "ENCODE cCRE, \ndistal enhancer", fontsize = 7, fontcolor = "darkgoldenrod2",
    rot = 0,
    params = params_a,
    y = 2.7, x = 0.1, height = 0.5,
) 

## Load ENCODE ChIP-seq: reps 1 and 2 from accession ENCFF167VRL http://dcc.blueprint-epigenome.eu/#/files
CTCF_encode <- readBigwig(file = "/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/external_data/ENCODE/ENCFF167VRL_CTCF.bigWig", 
                          chrom = mychrom,
                          chromstart = mystart,
                          chromend = myend)
CTCFsignal <- plotSignal(
    data = CTCF_encode,
    params = params_a, height = 1, y = 1,
    linecolor = "maroon3"
)
# text for ENCODE CTCF ChIP seq
plotText(
    label = "ENCODE CTCF, \nmono (-log10P)", fontsize = 7, fontcolor = "maroon3",
    rot = 0,
    params = params_a,
    y = 1.2, x = 0.1, height = 0.5,
) 

# range for CTCF
annoYaxis(
    CTCFsignal,
    at = c(0, 176),
    label = c(0, 176), # top -log10P val
    main = TRUE, 
    axisLine = T,
    params = params_a, 
    y = 1, height = 1, fontsize = 7, fontcolor = "maroon3"
)

## Annotate genome label
annoXaxis(plot = genes_a, params = params_a, fontsize = 7,
                   y = 4.2, at = c(24058000, 24058500, 24059000)
)


## going to add CTCF motif in inkscape
#CTCF_motif <- readPNG("~/HRJ_monocytes/paper/CTCF_rs7146599.png")

#plotRaster(
#    image = CTCF_motif,
#    params = params_a,
#    x = 3.2, y = 4, width = 5, height = 2, just = c("left", "top")
#)
dev.off()

In [ ]:
### old code. including geno specific and maxatac ctcf.
#################### ADD GENO SPECIFIC CHICAGO PLOTBAITS

REF <- fread(file = "/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/CHiC/chicago/combinations_output/5K-bins/genotype_specific_runs/14:24527672:G:A_homozygous_REF/14:24527672:G:A_homozygous_REFallReplicates_Chicago_135203.bg")
setnames(REF, "V4", "score")
CHiC_signal_REF <- plotSignal(
    data = REF, binSize = 50, binCap = F, range = c(0,52),
    params = params_a, height = 0.8, y = 4.5,
    linecolor = "#0072B2", fill = "#56B4E9"
)
annoYaxis(CHiC_signal_REF, at = NULL, label = T, main = TRUE, axisLine = T, params = params_a, y = 4.5, height = 0.5, fontsize = 7, fontcolor = "deepskyblue2")

ALT <- fread(file = "/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/CHiC/chicago/combinations_output/5K-bins/genotype_specific_runs/14:24527672:G:A_homozygous_ALT/14:24527672:G:A_homozygous_ALTallReplicates_Chicago_135203.bg")
setnames(ALT, "V4", "score")
CHiC_signal_ALT <- plotSignal(
    data = ALT, binSize = 50, binCap = F, range = c(0,52),
    params = params_a, height = 0.8, y = 5.5,
    linecolor = "#b2182b", fill = "#D55E00"
)
annoYaxis(CHiC_signal_ALT, at = NULL, label = T, main = TRUE, axisLine = T, params = params_a, y = 5.5, height = 0.5, fontsize = 7, fontcolor = "firebrick2")

REFP <- fread(file = "/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/CHiC/chicago/combinations_output/5K-bins/genotype_specific_runs/14:24527672:G:A_homozygous_REF/14:24527672:G:A_homozygous_REFallReplicates_Chicago_135211.bg")
setnames(REFP, "V4", "score")
CHiC_signal_REFP <- plotSignal(
    data = REFP, binSize = 50, binCap = F, range = c(0,52),
    params = params_a, height = 0.8, y = 6.5,
    linecolor = "#0072B2", fill = "#56B4E9"
)
annoYaxis(CHiC_signal_REFP, at = NULL, label = T, main = TRUE, axisLine = T, params = params_a, y = 6.5, height = 0.5, fontsize = 7, fontcolor = "deepskyblue2")

## Bring in chic bw data from a bait for the SNP
ALTP <- fread(file = "/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/CHiC/chicago/combinations_output/5K-bins/genotype_specific_runs/14:24527672:G:A_homozygous_ALT/14:24527672:G:A_homozygous_ALTallReplicates_Chicago_135211.bg")
setnames(ALTP, "V4", "score")
CHiC_signal_ALTP <- plotSignal(
    data = ALTP, binSize = 50, binCap = F, range = c(0,52),
    params = params_a, height = 0.8, y = 7.5,
    linecolor = "#b2182b", fill = "#D55E00"
)
annoYaxis(CHiC_signal_ALTP, at = NULL, label = T, main = TRUE, axisLine = T, params = params_a, y = 7.5, height = 0.5, fontsize = 7, fontcolor = "firebrick2")

# text for loops (5kb geno specific) 
plotText(
    label = "rs7146599 CHi-C", fontsize = 7, fontcolor = "black",
    rot = 90,
    params = params_a,
    y = 6.4, x = 0.3, height = 0.5,
)
plotText(
    label = "Hom REF \nN reads", fontsize = 7, fontcolor = "#67a9cf",
    rot = 0,
    params = params_a,
    y = 4.5, x = 0.6, height = 0.5,
)
plotText(
    label = "Hom ALT \nN reads", fontsize = 7, fontcolor = "firebrick2",
    rot = 0,
    params = params_a,
    y = 5.5, x = 0.6, height = 0.5,
)
plotText(
    label = "PCK2 CHi-C", fontsize = 7, fontcolor = "black",
    rot = 90,
    params = params_a,
    y = 8.2, x = 0.3, height = 0.5,
)
plotText(
    label = "Hom REF \nN reads", fontsize = 7, fontcolor = "#67a9cf",
    rot = 0,
    params = params_a,
    y = 6.5, x = 0.6, height = 0.5,
)
plotText(
    label = "Hom ALT \nN reads", fontsize = 7, fontcolor = "firebrick2",
    rot = 0,
    params = params_a,
    y = 7.5, x = 0.6, height = 0.5,
)




##### Below: uncomment if you want to plot loops instead.
## plot loops
#plotPairsArches(
#    data = both_eQTL_bpe,
#    params = params_a,
#    y = 4.5, height = 1.5,
#    fill = colorby("allele", palette = 
#                colorRampPalette(c("firebrick2", "deepskyblue2"))), #darkorchid1
#    linecolor = "fill", flip = TRUE
#)
# text for loops: eQTLs, all
#plotText(
#    label = "eQTL loops", fontsize = 7,
#    rot = 90,
#    params = params_a,
#    y = 5.5, x = 0.4, height = 0.5,
#)
# text for loops: eQTLs, ref
#plotText(
#    label = "Hom REF", fontsize = 7, fontcolor = "deepskyblue2",
#    rot = 0,
#    params = params_a,
#    y = 4.5, x = 1, height = 0.5,
#)
# text for loops: eQTLs, alt
#plotText(
#    label = "Hom ALT", fontsize = 7, fontcolor = "firebrick2",
#    rot = 0,
#    params = params_a,
#    y = 5, x = 1, height = 0.5,
#)

## plot loops
#plotPairsArches(
#    data = both_PCK2_bpe,
#    params = params_a,
#    y = 6, height = 1.5,
#    fill = colorby("allele", palette = 
#                colorRampPalette(c("firebrick2", "darkblue", "deepskyblue2"))), #darkorchid1
#    linecolor = "fill", flip = TRUE
#)
# text for loops: PCK2 promoter, all
#plotText(
#    label = "PCK2 loops", fontsize = 7,
#    rot =90,
#    params = params_a,
#    y = 7.3, x = 0.4, height = 0.5,
#)
# text for loops: PCK2 promoter, ref
#plotText(
#    label = "Hom REF", fontsize = 7, fontcolor = "deepskyblue2",
#    rot = 0,
#    params = params_a,
#    y = 6, x = 1, height = 0.5,
#)
# text for loops: PCK2 promoter, alt
#plotText(
#    label = "Hom ALT", fontsize = 7, fontcolor = "firebrick2",
#    rot = 0,
#    params = params_a,
#    y = 6.5, x = 1, height = 0.5,
#)
# text for loops: PCK2 promoter, both
#plotText(
#    label = "REF/ALT", fontsize = 7, fontcolor = "darkblue",
#    rot = 0,
#    params = params_a,
#    y = 7, x = 1, height = 0.5,
#)

### plot CTCF from maxATAC
atac_CTCF <- readBigwig(file = "/rds/general/project/lms-spivakov-analysis/live/HRJ_monocytes/ATAC-seq/maxatac/output/CTCF/CTCF.averaged.chr14/CTCF.predict.bw",
                        chrom = mychrom,
                        chromstart = mystart,
                        chromend = myend)
plotCTCF <- plotSignal(
    data = atac_CTCF, binCap = F, binSize = 1,
    params = params_main, height = 0.5, y = 1.3,
    linecolor = "darkorange1"
)
# text for CTCF (maxATAC)
plotText(
    label = "CTCF \nmaxATAC", fontsize = 7, fontcolor = "darkorange1",
    rot = 0,
    params = params_text,
    y = 1.3, x = 0.4, height = 0.5,
) 
# range for CTCF (maxATAC)
annoYaxis(
    plotCTCF, at = c(0,1),
    label = c(0, 1), # top maxATAC score
    main = TRUE, axisLine = T, params = params_main, 
    y = 1.3, height = 0.5, fontsize = 7, fontcolor = "darkorange1"
)